# Pertubation : Change of color 

To study the influence of color on image classification of fruits, we explore the HSV feature of an image. To create variation in the color of an image, we gradually change the hue value from 0 to 252 of the images. 

We then move all the images of same hue to one common folder and perform the image classification task using the CNN model and compare the results.

In [1]:
# Include the required libraries and mount the gdrive to read the files
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2

import os

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import warnings
# filter warnings
warnings.filterwarnings('ignore')


import os
print(os.listdir)

from google.colab import drive
drive.mount('/content/gdrive/')
import nltk
import re

<built-in function listdir>
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
from PIL import Image

# We define the calling function image_hue_change


def image_hue_change(input_img, hue_val):
    image_arr = np.array(input_img,float)
    image_hsv = image_rgb2hsv(image_arr)
    image_hsv[..., 0] = hue_val
    image_rgb = image_hsv2rgb(image_hsv)
    return Image.fromarray(image_rgb, 'RGB')
  
 # Next the sub functions are defined

def image_rgb2hsv(image_rgb):
  
      # We use numpy function colorsys.rgb_to_hsv function to convert image to hsv
    # R, G, B component are in range 0 and 255
    
    image_rgb = image_rgb.astype('float')
    image_hsv = np.zeros_like(image_rgb)
    
    # in case an RGBA array was passed, just copy the A channel
    image_hsv[..., 3:] = image_rgb[..., 3:]
    red, green, blue = image_rgb[..., 0], image_rgb[..., 1], image_rgb[..., 2]
    
    max_c = np.max(image_rgb[..., :3], axis=-1)
    min_c = np.min(image_rgb[..., :3], axis=-1)
    image_hsv[..., 2] = max_c
    image_mask = max_c != min_c
    image_hsv[image_mask, 1] = (max_c - min_c)[image_mask] / max_c[image_mask]
    
    red_chan = np.zeros_like(red)
    green_chan = np.zeros_like(green)
    blue_chan = np.zeros_like(blue)
    
    red_chan[image_mask] = (max_c - red)[image_mask] / (max_c - min_c)[image_mask]
    green_chan[image_mask] = (max_c - green)[image_mask] / (max_c - min_c)[image_mask]
    blue_chan[image_mask] = (max_c - blue)[image_mask] / (max_c - min_c)[image_mask]
    
    image_hsv[..., 0] = np.select(
        [red == max_c, green == max_c], [blue_chan - green_chan, 2.0 + red_chan - blue_chan], default=4.0 + green_chan - red_chan)
    image_hsv[..., 0] = (image_hsv[..., 0] / 5.0) % 1.0
    return image_hsv

def image_hsv2rgb(image_hsv):
  
    #we use OPENCV function colorsys.hsv_to_rgb to convert an hsv to rgb image
    
    # Hue and Saturation values aare between 0.0 to 1.0 in numpy array form 
    # Similarly the Value component of HSV shoulf be between 0.0 to 255.0
    
       
    image_rgb = np.empty_like(image_hsv) #returns empty array like image_hsv 
    image_rgb[..., 3:] = image_hsv[..., 3:]
    hue, sat, value = image_hsv[..., 0], image_hsv[..., 1], image_hsv[..., 2]
    
    img_i = (hue * 6.0).astype('uint8')
    img_f = (hue * 6.0) - img_i
    img_p = value * (1.0 - sat)
    img_q = value * (1.0 - sat * img_f)
    img_t = value * (1.0 - sat * (1.0 - img_f))
    img_i = img_i % 6
    
    conditions = [sat == 0.0, img_i == 1, img_i == 2, img_i == 3, img_i == 4, img_i == 5]
    image_rgb[..., 0] = np.select(conditions, [value, img_q, img_p, img_p, img_t, value], default=value)
    image_rgb[..., 1] = np.select(conditions, [value, value, value, img_q, img_p, img_p], default=img_t)
    image_rgb[..., 2] = np.select(conditions, [value, img_p, img_t, value, value, img_q], default=img_p)
    
    return image_rgb.astype('uint8')


# Path to the input images is given 

path = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test/Lemon/"
#path = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test/Orange/"
#path = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test/Limes/"
#path = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test/Grape White/"

# The logic is then applied to all the Test images: 

dirs = os.listdir( path )
for item in dirs:
  filename = path + item 
  basename, ext = os.path.splitext(filename)
  input_img = Image.open(filename).convert('RGB')
  for hue in np.linspace(0, 360, 20):#here we give range of hue to be divided in 20 divisons
       
    new_img = image_hue_change(input_img, hue/360.)
    
    # give file names to the modified images 
    out_name = '{}_hue{:02d}.jpg'.format(basename, int(hue))
    
    #the modified image is then saved in the same location as of original test images
    new_img.save(out_name) 

In [0]:
# Create separate folders for each hue

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        


createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue00')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue18')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue37')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue56')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue75')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue94')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue113')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue132')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue151')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue170')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue189')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue208')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue227')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue246')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue265')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue284')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue303')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue322')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue341')
             
createFolder('./My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue360')

In [0]:
# Move the images to repective folders based on Hue values

import shutil
import os
import fnmatch

source = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test/Lemon/"

destination000 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue00/"
destination018 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue18/"
destination037 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue37/"
destination056 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue56/"
destination075 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue75/"
destination094 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue94/"
destination113 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue113/"
destination132 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue132/"
destination151 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue151/"
destination170 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue170/"
destination189 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue189/"
destination208 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue208/"
destination227 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue227/"
destination246 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue246/"
destination265 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue265/"
destination284 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue284/"
destination303 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue303/"
destination322 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue322/"
destination341 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue341/"
destination360 = "/content/gdrive/My Drive/Colab Notebooks/fruits/Test_Hue/Lemon/Hue360/"


for files in os.listdir(source):
  if fnmatch.fnmatch(files, '*hue00.jpg'):
  
    shutil.move( source + files, destination000)
  elif fnmatch.fnmatch(files, '*hue18.jpg'):
    shutil.move( source + files, destination018)
  
  elif fnmatch.fnmatch(files, '*hue37.jpg'):
    shutil.move( source + files, destination037)
    
  elif fnmatch.fnmatch(files, '*hue56.jpg'):
    shutil.move( source + files, destination056)
    
  elif fnmatch.fnmatch(files, '*hue75.jpg'):
    shutil.move( source + files, destination075)
       
  elif fnmatch.fnmatch(files, '*hue94.jpg'):
    shutil.move( source + files, destination094)
    
  elif fnmatch.fnmatch(files, '*hue113.jpg'):
    shutil.move( source + files, destination113)
    
  elif fnmatch.fnmatch(files, '*hue132.jpg'):
    shutil.move( source + files, destination132)
  
  elif fnmatch.fnmatch(files, '*hue151.jpg'):
    shutil.move( source + files, destination151)
    
  elif fnmatch.fnmatch(files, '*hue170.jpg'):
    shutil.move( source + files, destination170)
    
  elif fnmatch.fnmatch(files, '*hue189.jpg'):
    shutil.move( source + files, destination189)
    
  elif fnmatch.fnmatch(files, '*hue208.jpg'):
    shutil.move( source + files, destination208)
    
  elif fnmatch.fnmatch(files, '*hue227.jpg'):
    shutil.move( source + files, destination227)
    
  elif fnmatch.fnmatch(files, '*hue246.jpg'):
    shutil.move( source + files, destination246)
    
  elif fnmatch.fnmatch(files, '*hue265.jpg'):
    shutil.move( source + files, destination265)
    
  elif fnmatch.fnmatch(files, '*hue284.jpg'):
    shutil.move( source + files, destination284)
    
  elif fnmatch.fnmatch(files, '*hue303.jpg'):
    shutil.move( source + files, destination303)
    
  elif fnmatch.fnmatch(files, '*hue322.jpg'):
    shutil.move( source + files, destination322)
    
  elif fnmatch.fnmatch(files, '*hue341.jpg'):
    shutil.move( source + files, destination341)
        
  elif fnmatch.fnmatch(files, '*hue360.jpg'):
    shutil.move( source + files, destination360)

Once the modified images are moved, the CNN model is run to evaluate the accuracy. 

This is repeated for all the desired fruit combinations such as lime and lemon , lemon and orange,cocos and kiwi etc.
